In [8]:
import math                       
import matplotlib.pyplot as plt   
import random                     
import numpy as np                
import numpy.linalg as lin        
import scipy.special as ss
import pickle
from cgi import test
from copy import deepcopy

In [9]:
def get_mag(A):
  return np.sum(A)

def distance(A,B):
  distance = sum(np.logical_and(A,B))
  return distance

def perm(A,N):
  return np.roll(A,N)

def density(A):
  num_1 = 0
  
  for i in range(len(A)):
    if A[i]==1:
      num_1 += 1
      
  return (num_1/len(A))*100

In [10]:
def u_gen_rand_hv(D,d):
    # Sanity checker
    if (D % 2):
        print("Error - D can't be an odd number")
        return 0
    # generate
    chosen = random.sample(range(D), k =int(D*d))
    rand_hv = np.array([1 if x in chosen else 0 for x in range(D)])
    return rand_hv

def create_item_mem(N,D,d):
  # insert nice code here
  item_mem = {}
  first = u_gen_rand_hv(D,d)
  item_mem.update({0:first})
  last = u_gen_rand_hv(D,d)
  item_mem.update({N-1:last})
  for i in range(1,N-1):
    temp_hv = np.concatenate((first[:int((D/N)*(N-i))],last[int((D/N)*(N-i)):])) 
    item_mem.update({i:temp_hv})    
  return item_mem

def bundle(block,t): # t - threshold
  if((len(block) % 2) == 0):
    print("Error! Block size should be odd.")
    return
  sum = np.sum(block,axis=0)
  bndl = np.where(sum>t,1,0)
  return bndl

In [11]:
def hdc_encode(voice,voice_im,D,d,Q=9, remove_idx = []): #d = density, Q - quntization steps
    voice = [round(x*10000) for x in voice]
    
    for x in range(len(voice)):
        if voice[x] > 7778:
            voice[x] = 0
        elif voice[x] > 5556:
            voice[x] = 1
        elif voice[x] > 3333:
            voice[x] = 2
        elif voice[x] > 1111:
            voice[x] = 3
        elif voice[x] > -1111:
            voice[x] = 4
        elif voice[x] > -3333:
            voice[x] = 5
        elif voice[x] > -5556:
            voice[x] = 6
        elif voice[x] > -7778:
            voice[x] = 7
        elif voice[x] > -10001:
            voice[x] = 8
            
    feature_hv_list = [voice_im[x] for x in voice]
    feature_hv_list = np.array([perm(feature_hv_list[x],x+1) for x in range(617)])
    
    threshold = 6 #first threshold
    bundle = np.sum(feature_hv_list,axis=0)

    #for threshold in range(1000):
    bndl = np.where(bundle>threshold,1,0)
    
    #    print(threshold,sum(bndl))
    return bndl

In [12]:
def redundancy_pruning(bin_reg):
  temp_list = []
  for i in bin_reg:
    temp_list.append(i)
    
  comp = enumerate(np.where(bin_reg[0]==bin_reg[1], bin_reg[0], "False"))
  remove = [i for i in comp if i[1]!="False"]
  temp_list = []
  return_rp = []
  
  for i in bin_reg[2::]:
    for j in remove:
      if int(j[1]) == int(i[int(j[0])]):
        temp_list.append(j)
    remove = temp_list
    temp_list = []
  
  for i in remove:
    return_rp.append(i[0])

  #delete the specified indices in all of the classes
  for i in range(len(bin_reg)):
      temp_list[i] = np.delete(bin_reg[i], return_rp)
  return_list = np.array(temp_list)
  
  return return_list, return_rp

In [13]:
D = 4096
Q = 9
d = 0.01
remove_idx = []

voice_im = np.zeros((9, D))

with open('im.txt', 'r') as fp:
    for i,line in enumerate(fp):
        for j in range(4096):
            voice_im[i][j] = int(line[j])
fp.close()
        
for i in range(9):
    print(i,distance(voice_im[0],voice_im[i]))

with open('isolet.pkl', 'rb') as f:
    isolet = pickle.load(f)
trainData, trainLabels, testData, testLabels = isolet
trainData = np.array(trainData)
testData = np.array(testData)

with open("testLabels.txt", "w") as tl:
    for i in testLabels:
        tl.write(str(i))
        tl.write("\n")
tl.close()

voice_am = dict()

0 40
1 36
2 26
3 25
4 20
5 16
6 12
7 7
8 2


In [14]:
non_bin_reg = np.zeros((26,D))
#print(hdc_encode(trainData[i],voice_im, D, d, Q))

density_all = 0
print(trainLabels)
print(len(trainLabels))
with open("sw_train_encoded_hvs.txt", "w") as ehv, open("trainLabels.txt", "w") as tl:
    for i in range(26):
        for index, value in enumerate(trainLabels):
            if value==i:
                encoded_hv = hdc_encode(trainData[index],voice_im, D, d, Q)
                density_one = density(np.array(encoded_hv))
                density_all += density_one
                non_bin_reg[trainLabels[index]] = np.add(non_bin_reg[trainLabels[index]], encoded_hv)
                
                for j in range(4096):
                    ehv.write(str(encoded_hv[j]))
                ehv.write("\n")
                
                tl.write(str(j))
                tl.write("\n")
ehv.close()

[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9,

In [ ]:
print("Encoded HVs Mean Density: ", str(density_all/len(trainLabels)))

Training Encoded HVs Mean Density:  38.27714837487977


In [ ]:
threshold2 = 90 #second threshold
for i in range(26):
    voice_am[i] = np.where(non_bin_reg[i]>threshold2,1,0)

with open("sw_class_hvs.txt", "w") as chv:
    for i in voice_am.keys():
        chv.write("Class " + str(i) + ": ")
        for j in range(4096):
            chv.write(str(voice_am[i][j]))
        chv.write("\n") 
chv.close()

class_density = 0
for i in voice_am.values():
    class_density += density(i)
print("Class HVs Mean Density: ", str((class_density/26)))

# bin_reg, remove_idx = redundancy_pruning(voice_am.values())
# for i in range(26):
#     voice_am[i] = bin_reg[i]

Class HVs Mean Density:  44.28429236778846


In [ ]:
def similarity_search(voice,voice_im,voice_am,D,d,Q, wrt):
  # insert nice code here
  sim_score = 0
  sim_letter = '0'
  test_hv = hdc_encode(voice,voice_im,D,d,Q, remove_idx)
  
  for i in range(4096):
    wrt.write(str(test_hv[i]))
  wrt.write("\n")
  
  for each in voice_am.items():
    #print(each)
    if (distance(test_hv,each[1])>sim_score):
      sim_score,sim_letter = distance(test_hv,each[1]), each[0] 
  return sim_letter, sim_score

In [ ]:
def test_model(test_data,correct_values,voice_im,voice_am,D,d,Q,print_flag):

  # Simply iterate through all elements in the clean_letters set
  score = 0
  test_len = len(test_data)
  
  wrt = open("sw_test_encoded_hvs.txt", "w")
  inf = open("sw_inferences.txt", "w")
  
  for i in range(test_len):
    sim_letter, sim_score = similarity_search(test_data[i],voice_im,voice_am,D,d,Q, wrt)
    inf.write(str(sim_letter))
    inf.write("\n")
    if sim_letter == correct_values[i]:
      score += 1
      if(print_flag):
        print("CORRECT prediction! sim_letter: ", sim_letter, " sim_score: ", str(sim_score))
    else:
      if(print_flag):
        print("WRONG prediction! sim_letter: " , sim_letter, " sim_score: ", str(sim_score))
  wrt.close()
  inf.close()

  print("Final accuracy is: %f" %((score/test_len)*100))
  return (score/test_len)*100

In [ ]:
test_model(testData,testLabels,voice_im,voice_am,D,d,Q,True)

CORRECT prediction! sim_letter:  0  sim_score:  1316
CORRECT prediction! sim_letter:  0  sim_score:  1339
WRONG prediction! sim_letter:  4  sim_score:  1374
CORRECT prediction! sim_letter:  1  sim_score:  1367
CORRECT prediction! sim_letter:  2  sim_score:  1366
CORRECT prediction! sim_letter:  2  sim_score:  1359
WRONG prediction! sim_letter:  19  sim_score:  1371
WRONG prediction! sim_letter:  15  sim_score:  1385
CORRECT prediction! sim_letter:  4  sim_score:  1375
CORRECT prediction! sim_letter:  4  sim_score:  1389
WRONG prediction! sim_letter:  18  sim_score:  1259
CORRECT prediction! sim_letter:  5  sim_score:  1240
WRONG prediction! sim_letter:  19  sim_score:  1373
WRONG prediction! sim_letter:  15  sim_score:  1376
CORRECT prediction! sim_letter:  7  sim_score:  1356
CORRECT prediction! sim_letter:  7  sim_score:  1388
CORRECT prediction! sim_letter:  8  sim_score:  1338
CORRECT prediction! sim_letter:  8  sim_score:  1296
CORRECT prediction! sim_letter:  9  sim_score:  1330
